In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

import os
import glob
import shutil
os.listdir()

['.config', 'drive', 'sample_data']

In [0]:
os.listdir("drive/My Drive/IE 492/kubi_preds")

['LR_2.csv',
 'naive_2.csv',
 'LR_2_7_w_dates.csv',
 'naive_7.csv',
 'LR_2_7.csv',
 'LR_7.csv',
 'ygur_rima_preds_datetime.csv',
 'eval_plots',
 'Cat1_48hours.csv',
 'Cat2_categorical_48hours.csv',
 'Cat1_24hours.csv',
 'Cat2_categorical_24hours.csv',
 'Keras_Basic_dense_24.csv',
 'Keras_Basic_dense_48.csv',
 'HW_26.05_15.17_48.csv',
 'HW_26.05_15.17_24.csv']

In [0]:
shutil.copy("drive/My Drive/IE 492/RealTimeConsumption_unholidayized.csv","./")
shutil.copy("drive/My Drive/IE 492/RealTimeConsumption-01122014-09032020.csv","./")
shutil.copy("drive/My Drive/IE 492/RealTimeConsumption_holiday.csv","./")
shutil.copy("drive/My Drive/IE 492/kubi_temp_humid_wind_data_20151231-2020-04-02.csv","./")

'./kubi_temp_humid_wind_data_20151231-2020-04-02.csv'

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
%matplotlib inline
from tqdm.notebook import tqdm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# eval func

In [0]:
!pip install graphviz
!pip install sklearn --upgrade
from sklearn.metrics import mean_squared_error,mean_squared_log_error,mean_absolute_error
from sklearn.tree import DecisionTreeRegressor,plot_tree

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

eval_df=[]
def eval_function(name_,y_true, y_pred,valid_df):
    """
    name_:kullanılan modelin ismi
    y_true:actual values
    y_pred:predicted values
    valid_df:modele verdiginiz sekliyle validation dataframe'i ve yanına datetime'ı
    """
    ap="drive/My Drive/colab_datas/IE 492/kubi_preds/eval_plots/"+name_
    if not os.path.exists(ap):
        os.mkdir(ap)
    rmse_=np.sqrt(mean_squared_error(y_true,y_pred))
    rmsle_=np.sqrt(mean_squared_log_error(y_true,y_pred))
    mae_=mean_absolute_error(y_true,y_pred)
    mape_=mean_absolute_percentage_error(y_true, y_pred)
    mapes_each_period=[mean_absolute_percentage_error(y_true[i],y_pred[i]) for i in range(len(y_true))]
    mapes_each_period=pd.DataFrame(mapes_each_period,columns=["mapes"])
    mapes_each_period["Datetime"]=valid_df["Datetime"].iloc[-len(mapes_each_period):]
    print("RMSE: ",rmse_)
    print("RMSLE: ",rmsle_)
    print("MAE: ",mae_)
    print("MAPE: ",mape_)
    fig=plt.figure(figsize=(16,10))
    sns.scatterplot(x=y_pred,y=y_true)
    fig.savefig("drive/My Drive/colab_datas/IE 492/kubi_preds/eval_plots/"+name_+"/actual_vs_predicted.png")
    plt.pause(0.01)
    fig=plt.figure(figsize=(16,10))
    sns.residplot(y_true,y_pred)
    plt.pause(0.01)
    fig.savefig("drive/My Drive/colab_datas/IE 492/kubi_preds/eval_plots/"+name_+"/residplot.png")
    nchunks=6 # plot count
    fig,ax=plt.subplots(nrows=32,figsize=(32,125))
    valid_copy=valid_df.reset_index(drop=True).copy()

    for i,datetime_ in enumerate(np.array_split(valid_copy["Datetime"],32)):
        sns.lineplot(y=y_true[datetime_.index.tolist()],x=datetime_,color="green",ax=ax[i])
        sns.lineplot(y=y_pred[datetime_.index.tolist()],x=datetime_,color="red",ax=ax[i])
    fig.savefig("drive/My Drive/colab_datas/IE 492/kubi_preds/eval_plots/"+name_+"/lineplot_pred_act_{}.png".format(i))

    fig,ax=plt.subplots(nrows=nchunks,figsize=(32,25))
    for i,datetime_ in enumerate(np.array_split(mapes_each_period,nchunks)):
        sns.lineplot(y="mapes",x="Datetime",data=datetime_,ax=ax[i])
    fig.savefig("drive/My Drive/colab_datas/IE 492/kubi_preds/eval_plots/"+name_+"/lineplot_errors.png")

    # plt.pause(0.01)
    # res_model=DecisionTreeRegressor(max_depth=4)
    # _, ax = plt.subplots(figsize=(60, 60))  # whatever size you want
    # fig=res_model.fit(valid_df.drop(columns=["Datetime"]),y_true-y_pred)
    # plot_tree(fig,ax=ax,feature_names=valid_df.drop(columns=["Datetime"]).columns,fontsize=16)
    # plt.pause(0.01)
    # ## saatlik mape tanımla
    return name_, rmse_,rmsle_,mae_,mape_

Requirement already up-to-date: sklearn in /usr/local/lib/python3.6/dist-packages (0.0)


# Consumption df

In [0]:
# # bruuuuuuuh
# cons_holiday_binary=pd.read_csv("RealTimeConsumption_holiday.csv",encoding='latin-1')
# cons_holiday_binary["Datetime"]=cons_holiday_binary["Date"]+" "+cons_holiday_binary["Hour"]
# cons_holiday_binary["Datetime"]=pd.to_datetime(cons_holiday_binary["Datetime"],format="%d.%m.%Y %H:%M")
# cons_holiday_binary.head()

In [0]:
# catboostu 1 aylık yap

In [0]:
preds_paths=glob.glob("drive/My Drive/IE 492/kubi_preds/*.csv")
preds_paths

['drive/My Drive/IE 492/kubi_preds/LR_2.csv',
 'drive/My Drive/IE 492/kubi_preds/naive_2.csv',
 'drive/My Drive/IE 492/kubi_preds/LR_2_7_w_dates.csv',
 'drive/My Drive/IE 492/kubi_preds/naive_7.csv',
 'drive/My Drive/IE 492/kubi_preds/LR_2_7.csv',
 'drive/My Drive/IE 492/kubi_preds/LR_7.csv',
 'drive/My Drive/IE 492/kubi_preds/ygur_rima_preds_datetime.csv',
 'drive/My Drive/IE 492/kubi_preds/Cat1_48hours.csv',
 'drive/My Drive/IE 492/kubi_preds/Cat2_categorical_48hours.csv',
 'drive/My Drive/IE 492/kubi_preds/Cat1_24hours.csv',
 'drive/My Drive/IE 492/kubi_preds/Cat2_categorical_24hours.csv',
 'drive/My Drive/IE 492/kubi_preds/Keras_Basic_dense_24.csv',
 'drive/My Drive/IE 492/kubi_preds/Keras_Basic_dense_48.csv',
 'drive/My Drive/IE 492/kubi_preds/HW_26.05_15.17_48.csv',
 'drive/My Drive/IE 492/kubi_preds/HW_26.05_15.17_24.csv']

In [0]:
#a=pd.read_csv("drive/My Drive/colab_datas/IE 492/kubi_preds/ygur_rima_preds_datetime.csv")
#a["Datetime"]=pd.to_datetime(a["Datetime"],format="%d/%m/%Y %H:%M")
#a=a.iloc[:,:-1]
#a.columns=["Datetime","Arima"]
#a.to_csv("drive/My Drive/colab_datas/IE 492/kubi_preds/ygur_rima_preds_datetime.csv",index=False)

In [0]:
#path_="drive/My Drive/colab_datas/IE 492/kubi_preds/erdem_HW_preds_new.csv"
#a=pd.read_csv(path_)
#a["Datetime"]=pd.to_datetime(a["Datetime"])
#a.loc[a["Datetime"]>=pd.to_datetime("2019-02-01 00:00:00")][["Datetime","HW_unhol_reg"]].to_csv(path_,index=False)

In [0]:
pred_dfs=[]
for i,path_ in enumerate(preds_paths):
    df=pd.read_csv(path_)
   # if "HW" in path_:
    #    df=df.loc[df["Datetime"].between('2019-02-01','2020-03-05')].reset_index(drop=True)
    #elif "rima" in path_:
    #    df["Datetime"]=pd.date_range('2019-02-01','2020-03-04',freq="H")[:-1]
    df.columns=["Datetime_"+str(i),path_.split("/")[-1].strip(".csv")]
    pred_dfs.append(df)
preds=pd.concat(pred_dfs,axis=1)
preds = preds.drop(columns = ["Cat1_24hour", "Cat2_categorical_24hour", "Keras_Basic_dense_24", "HW_26.05_15.17_24"])

In [0]:
preds=preds[["Datetime_0"]+[i for i in preds.columns if not i.startswith("Datetime")]].rename(columns={"Datetime_0":"Datetime"})
preds

,Datetime,LR_2,naive_2,LR_2_7_w_date,naive_7,LR_2_7,LR_7,ygur_rima_preds_datetime,Cat1_48hour,Cat2_categorical_48hour,Keras_Basic_dense_48,HW_26.05_15.17_48
0,2019-02-01 00:00:00,31744.801869,31427.68,31374.377264,32021.03,31872.179211,32126.739766,31948.68315,31312.281942,31368.284964,31493.430,30684.197615
1,2019-02-01 01:00:00,30353.806407,29554.05,29685.964658,30105.80,30060.616031,30503.032726,29979.44233,29321.171197,29342.353761,29617.531,27926.819198
2,2019-02-01 02:00:00,29429.213104,28308.65,28505.461085,28730.71,28791.507529,29337.249341,28720.68574,28112.010835,28313.998802,28455.043,26546.141624
3,2019-02-01 03:00:00,28904.205359,27601.48,27855.221933,27957.44,28075.676511,28681.681110,28046.44589,27611.726344,27380.407098,27712.234,25968.405327
4,2019-02-01 04:00:00,28714.921348,27346.52,27727.815649,27802.84,27896.662105,28550.613245,27738.71164,27145.591865,27480.197660,27759.875,25889.936371
...,...,...,...,...,...,...,...,...,...,...,...,...
9523,2020-03-03 19:00:00,34433.450000,35049.21,38763.630766,39283.70,37640.730230,38283.936762,39055.31804,36533.611538,37024.468681,38331.830,38763.053506
9524,2020-03-03 20:00:00,34324.041524,34901.84,38428.945844,38791.40,37280.720755,37866.571236,38426.28253,36400.613389,37347.225005,37396.150,38194.641721
9525,2020-03-03 21:00:00,33821.654912,34225.14,37491.797795,37588.86,36301.219739,36847.073493,37493.64405,35472.911196,36789.700181,36533.484,37232.506117
9526,2020-03-03 22:00:00,33102.945835,33257.06,36424.989624,36290.64,35168.705158,35746.459492,36200.27294,34919.122612,35609.855302,35341.290,35950.473223


In [0]:
shutil.copy("drive/My Drive/IE 492/Consumption_special_days_v2.csv","./")

'./Consumption_special_days_v2.csv'

In [0]:
# bruuuuuuuh
cons_holiday_binary=pd.read_csv("Consumption_special_days_v2.csv",encoding='latin-1')
cons_holiday_binary["Datetime"]=cons_holiday_binary["Date"]+" "+cons_holiday_binary["Hour"]
cons_holiday_binary["Datetime"]=pd.to_datetime(cons_holiday_binary["Datetime"],format="%d.%m.%Y %H:%M")
cons_holiday_binary.head()

,Date,Hour,Consumption,Datetime,is_religious_holiday,is_normal_holiday,is_special_day
0,31.12.2015,00:00,29590.88,2015-12-31 00:00:00,0,1,0
1,31.12.2015,01:00,27785.43,2015-12-31 01:00:00,0,1,0
2,31.12.2015,02:00,26516.53,2015-12-31 02:00:00,0,1,0
3,31.12.2015,03:00,26091.62,2015-12-31 03:00:00,0,1,0
4,31.12.2015,04:00,25871.54,2015-12-31 04:00:00,0,1,0


In [0]:
preds["reel"]=cons_holiday_binary.loc[cons_holiday_binary["Datetime"]>=pd.to_datetime("2019-02-01 00:00:00")][["Consumption"]].values
preds.Datetime=pd.to_datetime(preds.Datetime)
preds

,Datetime,LR_2,naive_2,LR_2_7_w_date,naive_7,LR_2_7,LR_7,ygur_rima_preds_datetime,Cat1_48hour,Cat2_categorical_48hour,Keras_Basic_dense_48,HW_26.05_15.17_48,reel
0,2019-02-01 00:00:00,31744.801869,31427.68,31374.377264,32021.03,31872.179211,32126.739766,31948.68315,31312.281942,31368.284964,31493.430,30684.197615,31878.29
1,2019-02-01 01:00:00,30353.806407,29554.05,29685.964658,30105.80,30060.616031,30503.032726,29979.44233,29321.171197,29342.353761,29617.531,27926.819198,30064.08
2,2019-02-01 02:00:00,29429.213104,28308.65,28505.461085,28730.71,28791.507529,29337.249341,28720.68574,28112.010835,28313.998802,28455.043,26546.141624,28813.77
3,2019-02-01 03:00:00,28904.205359,27601.48,27855.221933,27957.44,28075.676511,28681.681110,28046.44589,27611.726344,27380.407098,27712.234,25968.405327,27865.76
4,2019-02-01 04:00:00,28714.921348,27346.52,27727.815649,27802.84,27896.662105,28550.613245,27738.71164,27145.591865,27480.197660,27759.875,25889.936371,27554.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9523,2020-03-03 19:00:00,34433.450000,35049.21,38763.630766,39283.70,37640.730230,38283.936762,39055.31804,36533.611538,37024.468681,38331.830,38763.053506,37936.79
9524,2020-03-03 20:00:00,34324.041524,34901.84,38428.945844,38791.40,37280.720755,37866.571236,38426.28253,36400.613389,37347.225005,37396.150,38194.641721,37553.37
9525,2020-03-03 21:00:00,33821.654912,34225.14,37491.797795,37588.86,36301.219739,36847.073493,37493.64405,35472.911196,36789.700181,36533.484,37232.506117,36581.73
9526,2020-03-03 22:00:00,33102.945835,33257.06,36424.989624,36290.64,35168.705158,35746.459492,36200.27294,34919.122612,35609.855302,35341.290,35950.473223,35434.36


In [0]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [0]:
mapes={}
calc_mapes=preds.set_index("Datetime")
for model_name in calc_mapes.columns[:-1]:
    y_true,y_pred=calc_mapes[["reel",model_name]].values.T
    mape_=mean_absolute_percentage_error(y_true,y_pred)
    mapes[model_name]=mape_
pd.DataFrame([list(mapes.keys()),list(mapes.values())]).T.sort_values(1)

,0,1
7,Cat1_48hour,2.99118
8,Cat2_categorical_48hour,3.24907
6,ygur_rima_preds_datetime,3.46533
9,Keras_Basic_dense_48,3.68307
10,HW_26.05_15.17_48,3.89177
2,LR_2_7_w_date,4.05324
4,LR_2_7,4.98923
3,naive_7,5.20805
5,LR_7,5.22779
0,LR_2,7.85569


In [0]:
## 5 iq move düzelt bunu
best_models=[]
for i,row in preds.iterrows():
    best_mape=999999
    best_model=""
    for col in preds.columns.difference(["Datetime","reel"]):
        mape_=mean_absolute_percentage_error(row["reel"],row[col])
        if best_mape>mape_:
            best_mape=mape_
            best_model=col
    best_models.append(best_model)
preds["best_model"]=best_models
len(preds)+1

9529

In [0]:
mapes["best_case"]=preds.loc[preds["best_model"]!="unknown"].apply(lambda x: mean_absolute_percentage_error(x["reel"],x[x["best_model"]]),axis=1).mean()

In [0]:
mapes

{'Cat1_48hour': 2.991182739400095,
 'Cat2_categorical_48hour': 3.2490686862665186,
 'HW_26.05_15.17_48': 3.89177361329798,
 'Keras_Basic_dense_48': 3.683070359734644,
 'LR_2': 7.855694573000102,
 'LR_2_7': 4.989226864853979,
 'LR_2_7_w_date': 4.053236198257791,
 'LR_7': 5.22779264475642,
 'best_case': 0.8467312948835701,
 'naive_2': 8.15891679451568,
 'naive_7': 5.208054715113833,
 'ygur_rima_preds_datetime': 3.4653294353157422}

In [0]:
def brute_meta(n,buffer):
    shift_timespan=n*24
    chosen_models=["unknown"]*(shift_timespan+buffer)
    print("meta brute with n = {}, buffer = {}".format(n,buffer))
    for i in tqdm(range(shift_timespan,len(preds))):
        ## buraya prob koyacaggız
        chosen_models.append(preds["best_model"].iloc[i-shift_timespan:i].value_counts().index[0])
    preds["chosen"]=chosen_models[:len(chosen_models)-buffer]
    mapes_brute=preds.loc[preds["chosen"]!="unknown"].apply(lambda x: mean_absolute_percentage_error(x["reel"],x[x["chosen"]]),axis=1)
    mapes["window_{}_days_{}_buffer".format(n,buffer)]=mapes_brute.mean()
    print(mapes_brute.mean())

In [0]:
combins=np.concatenate([[(j,i) for i in [0,24,48]] for j in [5,7,14,28,30]])
for n,buffer in combins:
    brute_meta(n,buffer)

meta brute with n = 5, buffer = 0



3.5275426330513726
meta brute with n = 5, buffer = 24



3.9594618504691304
meta brute with n = 5, buffer = 48



3.8546868331000166
meta brute with n = 7, buffer = 0



3.204682277362977
meta brute with n = 7, buffer = 24


...


3.2656800115992284
meta brute with n = 28, buffer = 48



3.283781768044813
meta brute with n = 30, buffer = 0



3.2010163308281365
meta brute with n = 30, buffer = 24



3.257057595210381
meta brute with n = 30, buffer = 48



3.229901379904504


In [0]:
mapes

{'Cat1_48hour': 2.991182739400095,
 'Cat2_categorical_48hour': 3.2490686862665186,
 'HW_26.05_15.17_48': 3.89177361329798,
 'Keras_Basic_dense_48': 3.683070359734644,
 'LR_2': 7.855694573000102,
 'LR_2_7': 4.989226864853979,
 'LR_2_7_w_date': 4.053236198257791,
 'LR_7': 5.22779264475642,
 'best_case': 0.8467312948835701,
 'naive_2': 8.15891679451568,
 'naive_7': 5.208054715113833,
 'window_14_days_0_buffer': 3.159421816731837,
 'window_14_days_24_buffer': 3.3311105076450755,
 'window_14_days_48_buffer': 3.3297741234672342,
 'window_28_days_0_buffer': 3.182952702033788,
 'window_28_days_24_buffer': 3.2656800115992284,
 'window_28_days_48_buffer': 3.283781768044813,
 'window_30_days_0_buffer': 3.2010163308281365,
 'window_30_days_24_buffer': 3.257057595210381,
 'window_30_days_48_buffer': 3.229901379904504,
 'window_5_days_0_buffer': 3.5275426330513726,
 'window_5_days_24_buffer': 3.9594618504691304,
 'window_5_days_48_buffer': 3.8546868331000166,
 'window_7_days_0_buffer': 3.20468227736

In [0]:
nisan_24=preds.loc[preds["Datetime"].between(pd.to_datetime("2019-04-24 00:00:00"),pd.to_datetime("2019-04-25 00:00:00"))]
day_by_day_sort={}
for i,row in nisan_24.iterrows():
    mapes_a={}
    for col in nisan_24.columns[1:11]:
        mape_=mean_absolute_percentage_error(row["reel"],row[col])
        mapes_a[col]=mape_
    day_by_day_sort[row["Datetime"]]=mapes_a
day_by_day_sort

{Timestamp('2019-04-24 00:00:00'): {'Cat1_48hour': 3.2550492638705992,
  'Cat2_categorical_48hour': 4.0141117858170565,
  'Keras_Basic_dense_48': 7.213321168131337,
  'LR_2': 1.8654968712021158,
  'LR_2_7': 4.078852493495687,
  'LR_2_7_w_date': 3.7699078135235085,
  'LR_7': 6.957455676398823,
  'naive_2': 1.3105580265177919,
  'naive_7': 6.193579003332738,
  'ygur_rima_preds_datetime': 6.636509101465669},
 Timestamp('2019-04-24 01:00:00'): {'Cat1_48hour': 4.149418695961482,
  'Cat2_categorical_48hour': 3.9536312716998854,
  'Keras_Basic_dense_48': 7.046823417381996,
  'LR_2': 2.948258726195324,
  'LR_2_7': 4.214070795705579,
  'LR_2_7_w_date': 4.332855355359548,
  'LR_7': 7.993531180302443,
  'naive_2': 2.02019839323456,
  'naive_7': 6.291836117680087,
  'ygur_rima_preds_datetime': 6.5047205108557495},
 Timestamp('2019-04-24 02:00:00'): {'Cat1_48hour': 3.224480103978185,
  'Cat2_categorical_48hour': 3.527110735605522,
  'Keras_Basic_dense_48': 5.718579274491101,
  'LR_2': 3.50261943950

In [0]:
pd.DataFrame.from_dict(day_by_day_sort).to_csv("24_nisan_sorted.csv")

In [0]:
{key:pd.DataFrame.from_dict(day_by_day_sort[key],orient="index").sort_values(by=0).index.tolist() for key in day_by_day_sort.keys() }

{Timestamp('2019-04-24 00:00:00'): ['naive_2',
  'LR_2',
  'Cat1_48hour',
  'LR_2_7_w_date',
  'Cat2_categorical_48hour',
  'LR_2_7',
  'naive_7',
  'ygur_rima_preds_datetime',
  'LR_7',
  'Keras_Basic_dense_48'],
 Timestamp('2019-04-24 01:00:00'): ['naive_2',
  'LR_2',
  'Cat2_categorical_48hour',
  'Cat1_48hour',
  'LR_2_7',
  'LR_2_7_w_date',
  'naive_7',
  'ygur_rima_preds_datetime',
  'Keras_Basic_dense_48',
  'LR_7'],
 Timestamp('2019-04-24 02:00:00'): ['naive_2',
  'Cat1_48hour',
  'LR_2',
  'Cat2_categorical_48hour',
  'LR_2_7',
  'LR_2_7_w_date',
  'naive_7',
  'ygur_rima_preds_datetime',
  'Keras_Basic_dense_48',
  'LR_7'],
 Timestamp('2019-04-24 03:00:00'): ['Cat1_48hour',
  'naive_2',
  'Cat2_categorical_48hour',
  'LR_2_7',
  'LR_2',
  'LR_2_7_w_date',
  'Keras_Basic_dense_48',
  'ygur_rima_preds_datetime',
  'naive_7',
  'LR_7'],
 Timestamp('2019-04-24 04:00:00'): ['Cat1_48hour',
  'naive_2',
  'Cat2_categorical_48hour',
  'LR_2_7',
  'LR_2_7_w_date',
  'LR_2',
  'naive_7

In [0]:
pd.DataFrame.from_dict({key:pd.DataFrame.from_dict(day_by_day_sort[key],orient="index").sort_values(by=0).index.tolist() for key in day_by_day_sort.keys() }).T

,0,1,2,3,4,5,6,7,8,9
2019-04-24 00:00:00,naive_2,LR_2,Cat1_48hour,LR_2_7_w_date,Cat2_categorical_48hour,LR_2_7,naive_7,ygur_rima_preds_datetime,LR_7,Keras_Basic_dense_48
2019-04-24 01:00:00,naive_2,LR_2,Cat2_categorical_48hour,Cat1_48hour,LR_2_7,LR_2_7_w_date,naive_7,ygur_rima_preds_datetime,Keras_Basic_dense_48,LR_7
2019-04-24 02:00:00,naive_2,Cat1_48hour,LR_2,Cat2_categorical_48hour,LR_2_7,LR_2_7_w_date,naive_7,ygur_rima_preds_datetime,Keras_Basic_dense_48,LR_7
2019-04-24 03:00:00,Cat1_48hour,naive_2,Cat2_categorical_48hour,LR_2_7,LR_2,LR_2_7_w_date,Keras_Basic_dense_48,ygur_rima_preds_datetime,naive_7,LR_7
2019-04-24 04:00:00,Cat1_48hour,naive_2,Cat2_categorical_48hour,LR_2_7,LR_2_7_w_date,LR_2,naive_7,Keras_Basic_dense_48,ygur_rima_preds_datetime,LR_7
2019-04-24 05:00:00,Cat1_48hour,naive_2,LR_2_7,LR_2,Cat2_categorical_48hour,LR_2_7_w_date,naive_7,ygur_rima_preds_datetime,Keras_Basic_dense_48,LR_7
2019-04-24 06:00:00,naive_2,Cat1_48hour,LR_2_7,Cat2_categorical_48hour,LR_2,LR_2_7_w_date,naive_7,Keras_Basic_dense_48,ygur_rima_preds_datetime,LR_7
2019-04-24 07:00:00,naive_2,Cat1_48hour,Cat2_categorical_48hour,LR_2,LR_2_7,Keras_Basic_dense_48,LR_2_7_w_date,ygur_rima_preds_datetime,naive_7,LR_7
2019-04-24 08:00:00,Cat2_categorical_48hour,Cat1_48hour,Keras_Basic_dense_48,LR_2,naive_2,LR_2_7,LR_2_7_w_date,LR_7,ygur_rima_preds_datetime,naive_7
2019-04-24 09:00:00,Cat2_categorical_48hour,Cat1_48hour,LR_2_7_w_date,LR_2_7,naive_2,Keras_Basic_dense_48,LR_7,ygur_rima_preds_datetime,LR_2,naive_7


In [0]:
mapes

{'Cat1_48hour': 2.991182739400095,
 'Cat2_categorical_48hour': 3.2490686862665186,
 'HW_26.05_15.17_48': 3.89177361329798,
 'Keras_Basic_dense_48': 3.683070359734644,
 'LR_2': 7.855694573000102,
 'LR_2_7': 4.989226864853979,
 'LR_2_7_w_date': 4.053236198257791,
 'LR_7': 5.22779264475642,
 'best_case': 0.8467312948835701,
 'naive_2': 8.15891679451568,
 'naive_7': 5.208054715113833,
 'window_14_days_0_buffer': 3.159421816731837,
 'window_14_days_24_buffer': 3.3311105076450755,
 'window_14_days_48_buffer': 3.3297741234672342,
 'window_28_days_0_buffer': 3.182952702033788,
 'window_28_days_24_buffer': 3.2656800115992284,
 'window_28_days_48_buffer': 3.283781768044813,
 'window_30_days_0_buffer': 3.2010163308281365,
 'window_30_days_24_buffer': 3.257057595210381,
 'window_30_days_48_buffer': 3.229901379904504,
 'window_5_days_0_buffer': 3.5275426330513726,
 'window_5_days_24_buffer': 3.9594618504691304,
 'window_5_days_48_buffer': 3.8546868331000166,
 'window_7_days_0_buffer': 3.20468227736

In [0]:
pd.DataFrame.from_dict(mapes,orient="index").to_excel("Mapes.xlsx")

In [0]:
# preds.loc[1344,"Datetime"]=pd.to_datetime("2019-03-29 00:00:00")

## decision tree

In [0]:
shutil.copy("drive/My Drive/IE 492/Consumption_special_days_v2.csv","./")
cons_holiday_binary  = pd.read_csv('Consumption_special_days_v2.csv')
cons_holiday_binary["Datetime"]=cons_holiday_binary["Date"]+" "+cons_holiday_binary["Hour"]
cons_holiday_binary["Datetime"]=pd.to_datetime(cons_holiday_binary["Datetime"],format="%d.%m.%Y %H:%M")
cons_holiday_binary.head()


,Date,Hour,Consumption,Datetime,is_religious_holiday,is_normal_holiday,is_special_day
0,31.12.2015,00:00,29590.88,2015-12-31 00:00:00,0,1,0
1,31.12.2015,01:00,27785.43,2015-12-31 01:00:00,0,1,0
2,31.12.2015,02:00,26516.53,2015-12-31 02:00:00,0,1,0
3,31.12.2015,03:00,26091.62,2015-12-31 03:00:00,0,1,0
4,31.12.2015,04:00,25871.54,2015-12-31 04:00:00,0,1,0


In [0]:
weather_data=pd.read_csv('kubi_temp_humid_wind_data_20151231-2020-04-02.csv')
weather_data=weather_data.rename(columns={"rounded_date":"Datetime"})
weather_data["Datetime"]=pd.to_datetime(weather_data["Datetime"])
cons_holiday_binary=pd.merge(cons_holiday_binary,weather_data,on="Datetime",how="left")
cat_df_1=cons_holiday_binary[["Datetime"] + [i for i in cons_holiday_binary.columns if i.startswith("is_")]].copy()
cat_df_1["Hour"]=cat_df_1["Datetime"].apply(lambda x:x.hour).astype(int)
cat_df_1["Day"]=cat_df_1["Datetime"].apply(lambda x:x.day).astype(int)
cat_df_1["Month"]=cat_df_1["Datetime"].apply(lambda x:x.month).astype(int)
cat_df_1["weekday"]=cat_df_1["Datetime"].apply(lambda x: x.weekday()).astype(int)
cat_df_1.head()

,Datetime,is_religious_holiday,is_normal_holiday,is_special_day,Hour,Day,Month,weekday
0,2015-12-31 00:00:00,0,1,0,0,31,12,3
1,2015-12-31 01:00:00,0,1,0,1,31,12,3
2,2015-12-31 02:00:00,0,1,0,2,31,12,3
3,2015-12-31 03:00:00,0,1,0,3,31,12,3
4,2015-12-31 04:00:00,0,1,0,4,31,12,3


In [0]:
np.unique([i.split("_")[1] for i in weather_data.columns if i!="Datetime"])

array(['Adana / Incirlik', 'Adana / Sakirpasa', 'Afyon', 'Akhisar',
       'Ankara / Esenboga', 'Ankara / Etimesgut', 'Antalya', 'Aydin',
       'Balikesir', 'Bandirma', 'Batman', 'Bodrum', 'Canakkale',
       'Dalaman', 'Diyarbakir', 'Elazig', 'Erzincan', 'Erzurum',
       'Eskisehir', 'Gaziantep', 'Iskenderun', 'Isparta',
       'Isparta / Gölba??', 'Istanbul / Ataturk',
       'Izmir / Adnan Menderes', 'Izmir / Cigli', 'Kars',
       'Kayseri / Erkilet', 'Konya', 'Malatya / Erhac', 'Merzifon',
       'Murted Tur-Afb', 'Sivas', 'Tokat', 'Trabzon', 'Usak', 'Van',
       'Zonguldak'], dtype='<U22')

##DECISION TREE farklı değişkenlerle

In [0]:
preds['var_models'] = preds.iloc[:,1:11].var(axis=1)
preds['mean_models'] = preds.iloc[:,1:11].mean(axis=1)
preds['index']=range(0,len(preds))
preds

,Datetime,LR_2,naive_2,LR_2_7_w_date,naive_7,LR_2_7,LR_7,ygur_rima_preds_datetime,Cat1_48hour,Cat2_categorical_48hour,Keras_Basic_dense_48,HW_26.05_15.17_48,reel,best_model,chosen,var_models,mean_models,index
0,2019-02-01 00:00:00,31744.801869,31427.68,31374.377264,32021.03,31872.179211,32126.739766,31948.68315,31312.281942,31368.284964,31493.430,30684.197615,31878.29,LR_2_7,unknown,9.469347e+04,31668.948816,0
1,2019-02-01 01:00:00,30353.806407,29554.05,29685.964658,30105.80,30060.616031,30503.032726,29979.44233,29321.171197,29342.353761,29617.531,27926.819198,30064.08,LR_2_7,unknown,1.680739e+05,29852.376811,1
2,2019-02-01 02:00:00,29429.213104,28308.65,28505.461085,28730.71,28791.507529,29337.249341,28720.68574,28112.010835,28313.998802,28455.043,26546.141624,28813.77,LR_2_7,unknown,1.871761e+05,28670.452944,2
3,2019-02-01 03:00:00,28904.205359,27601.48,27855.221933,27957.44,28075.676511,28681.681110,28046.44589,27611.726344,27380.407098,27712.234,25968.405327,27865.76,LR_2_7_w_date,unknown,2.318003e+05,27982.651825,3
4,2019-02-01 04:00:00,28714.921348,27346.52,27727.815649,27802.84,27896.662105,28550.613245,27738.71164,27145.591865,27480.197660,27759.875,25889.936371,27554.48,Cat2_categorical_48hour,unknown,2.393240e+05,27816.374851,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9523,2020-03-03 19:00:00,34433.450000,35049.21,38763.630766,39283.70,37640.730230,38283.936762,39055.31804,36533.611538,37024.468681,38331.830,38763.053506,37936.79,LR_2_7,ygur_rima_preds_datetime,2.784199e+06,37439.988602,9523
9524,2020-03-03 20:00:00,34324.041524,34901.84,38428.945844,38791.40,37280.720755,37866.571236,38426.28253,36400.613389,37347.225005,37396.150,38194.641721,37553.37,Keras_Basic_dense_48,ygur_rima_preds_datetime,2.242154e+06,37116.379028,9524
9525,2020-03-03 21:00:00,33821.654912,34225.14,37491.797795,37588.86,36301.219739,36847.073493,37493.64405,35472.911196,36789.700181,36533.484,37232.506117,36581.73,Keras_Basic_dense_48,ygur_rima_preds_datetime,1.801375e+06,36256.548537,9525
9526,2020-03-03 22:00:00,33102.945835,33257.06,36424.989624,36290.64,35168.705158,35746.459492,36200.27294,34919.122612,35609.855302,35341.290,35950.473223,35434.36,Keras_Basic_dense_48,ygur_rima_preds_datetime,1.381039e+06,35206.134096,9526


Index, model variance, model mean

In [0]:
from sklearn.tree import DecisionTreeClassifier

shift_timespan=30*24
primary_predicted_model=["unknown"]*shift_timespan
for i in tqdm(range(shift_timespan,len(preds))):
    train_df=preds[["Datetime","best_model",'var_models','mean_models','index']].iloc[:i+48]
    valid_df=train_df.iloc[-1]
    train_df=train_df.iloc[:i]
    tree=DecisionTreeClassifier()
    tree.fit(train_df.drop(columns=["best_model","Datetime"]),train_df["best_model"])
    # random forestt rreggrressor
    pred=tree.predict(pd.DataFrame(valid_df).T.drop(columns=["best_model","Datetime"]))
    # predict proba
    primary_predicted_model.append(pred[0])
preds["primary_tree_predicted"]=primary_predicted_model
mapes=preds.loc[preds["primary_tree_predicted"]!="unknown"].apply(lambda x: mean_absolute_percentage_error(x["reel"],x[x["primary_tree_predicted"]]),axis=1)
mapes.mean()

2.6722551975150037

In [0]:
cat_df_1['Hour'] = cat_df_1['Hour'].astype("category")
cat_df_1['Day'] = cat_df_1['Day'].astype("category")
cat_df_1['Month'] = cat_df_1['Month'].astype("category")
cat_df_1['weekday'] = cat_df_1['weekday'].astype("category")
cat_df_1['is_religious_holiday'] = cat_df_1['is_religious_holiday'].astype("category")
cat_df_1['is_normal_holiday'] = cat_df_1['is_normal_holiday'].astype("category")
cat_df_1['is_special_day'] = cat_df_1['is_special_day'].astype("category")


In [0]:
cat_df_1

,Datetime,is_religious_holiday,is_normal_holiday,is_special_day,Hour,Day,Month,weekday
0,2015-12-31 00:00:00,0,1,0,0,31,12,3
1,2015-12-31 01:00:00,0,1,0,1,31,12,3
2,2015-12-31 02:00:00,0,1,0,2,31,12,3
3,2015-12-31 03:00:00,0,1,0,3,31,12,3
4,2015-12-31 04:00:00,0,1,0,4,31,12,3
...,...,...,...,...,...,...,...,...
36571,2020-03-03 19:00:00,0,0,0,19,3,3,1
36572,2020-03-03 20:00:00,0,0,0,20,3,3,1
36573,2020-03-03 21:00:00,0,0,0,21,3,3,1
36574,2020-03-03 22:00:00,0,0,0,22,3,3,1


Index, model variance, model mean, date, special days

In [0]:
predicted_model_v2=["unknown"]*shift_timespan
for i in tqdm(range(shift_timespan,len(preds))):
    train_df=preds[["Datetime","best_model",'var_models','mean_models','index']].iloc[:i+48]
    train_df=pd.merge(train_df,cat_df_1,how="left",on="Datetime")
    valid_df=train_df.iloc[-1]
    train_df=train_df.iloc[:i]
    tree=DecisionTreeClassifier()
    tree.fit(train_df.drop(columns=["best_model","Datetime"]),train_df["best_model"])
    # random forestt rreggrressor
    pred=tree.predict(pd.DataFrame(valid_df).T.drop(columns=["best_model","Datetime"]))
    # predict proba
    predicted_model_v2.append(pred[0])
preds["tree_predicted_2"]=predicted_model_v2
mapes=preds.loc[preds["tree_predicted_2"]!="unknown"].apply(lambda x: mean_absolute_percentage_error(x["reel"],x[x["tree_predicted_2"]]),axis=1)
mapes.mean()

3.9607501061701176

Weather ekleme

In [0]:
shutil.copy("drive/My Drive/IE 492/weather_turkey_processed.zip","./")
!unzip weather_turkey_processed.zip

Archive:  weather_turkey_processed.zip
   creating: weather_turkey_processed/
  inflating: weather_turkey_processed/36_26.5.csv  
  inflating: weather_turkey_processed/38.5_42.csv  
  inflating: weather_turkey_processed/36_26.csv  
  inflating: weather_turkey_processed/36_27.5.csv  
  inflating: weather_turkey_processed/36_27.csv  
  inflating: weather_turkey_processed/36_28.5.csv  
  inflating: weather_turkey_processed/36_28.csv  
  inflating: weather_turkey_processed/36_29.5.csv  
  inflating: weather_turkey_processed/36_29.csv  
  inflating: weather_turkey_processed/36_30.5.csv  
  inflating: weather_turkey_processed/36_30.csv  
  inflating: weather_turkey_processed/36_31.5.csv  
  inflating: weather_turkey_processed/36_31.csv  
  inflating: weather_turkey_processed/36_32.5.csv  
  inflating: weather_turkey_processed/36_32.csv  
  inflating: weather_turkey_processed/36_33.5.csv  
  inflating: weather_turkey_processed/36_33.csv  
  inflating: weather_turkey_processed/36_34.5.csv  
  

In [0]:
weathers=[]
for path in tqdm(glob.glob("weather_turkey_processed/*.csv")):
    weathers.append(pd.read_csv(path)[['forecast_epoch','lon','lat','TMP_2.m.above.ground']])
weathers=pd.concat(weathers)
weathers["lon_lat"]=weathers["lon"].astype(str)+"_"+weathers["lat"].astype(str)
weathers_pivoted=pd.pivot_table(weathers,index=["forecast_epoch"],columns=["lon_lat"],values=["TMP_2.m.above.ground"])
weathers_pivoted.columns=["temp-"+i[1] for i in weathers_pivoted.columns]

In [0]:
weathers_pivoted

,temp-26.0_36.0,temp-26.0_36.5,temp-26.0_37.0,temp-26.0_37.5,temp-26.0_38.0,temp-26.0_38.5,temp-26.0_39.0,temp-26.0_39.5,temp-26.0_40.0,temp-26.0_40.5,temp-26.0_41.0,temp-26.0_41.5,temp-26.0_42.0,temp-26.5_36.0,temp-26.5_36.5,temp-26.5_37.0,temp-26.5_37.5,temp-26.5_38.0,temp-26.5_38.5,temp-26.5_39.0,temp-26.5_39.5,temp-26.5_40.0,temp-26.5_40.5,temp-26.5_41.0,temp-26.5_41.5,temp-26.5_42.0,temp-27.0_36.0,temp-27.0_36.5,temp-27.0_37.0,temp-27.0_37.5,temp-27.0_38.0,temp-27.0_38.5,temp-27.0_39.0,temp-27.0_39.5,temp-27.0_40.0,temp-27.0_40.5,temp-27.0_41.0,temp-27.0_41.5,temp-27.0_42.0,temp-27.5_36.0,...,temp-43.5_42.0,temp-44.0_36.0,temp-44.0_36.5,temp-44.0_37.0,temp-44.0_37.5,temp-44.0_38.0,temp-44.0_38.5,temp-44.0_39.0,temp-44.0_39.5,temp-44.0_40.0,temp-44.0_40.5,temp-44.0_41.0,temp-44.0_41.5,temp-44.0_42.0,temp-44.5_36.0,temp-44.5_36.5,temp-44.5_37.0,temp-44.5_37.5,temp-44.5_38.0,temp-44.5_38.5,temp-44.5_39.0,temp-44.5_39.5,temp-44.5_40.0,temp-44.5_40.5,temp-44.5_41.0,temp-44.5_41.5,temp-44.5_42.0,temp-45.0_36.0,temp-45.0_36.5,temp-45.0_37.0,temp-45.0_37.5,temp-45.0_38.0,temp-45.0_38.5,temp-45.0_39.0,temp-45.0_39.5,temp-45.0_40.0,temp-45.0_40.5,temp-45.0_41.0,temp-45.0_41.5,temp-45.0_42.0
forecast_epoch,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1421290800,288.150,287.920,287.420,287.140,287.070,279.610,285.980,285.220,284.710,283.380,274.390,271.950,271.530,288.290,287.950,287.670,287.490,287.020,280.290,285.160,282.460,279.520,281.070,275.000,272.090,271.620,288.340,287.990,287.250,287.190,285.110,282.580,281.120,279.840,275.930,281.470,274.890,273.750,271.350,288.170,...,272.010,278.320,277.070,273.060,258.400,260.400,256.750,254.020,262.980,270.890,253.430,259.890,268.010,273.540,277.360,273.610,270.210,258.780,262.020,255.440,257.030,264.980,273.100,256.540,267.710,270.890,272.800,276.790,265.210,262.620,268.540,270.450,265.860,268.000,272.890,255.180,265.820,269.650,273.050,265.630
1421301600,288.100,288.040,287.570,287.360,287.440,279.800,286.390,285.520,285.190,284.180,274.770,272.770,272.900,288.240,288.040,287.710,287.470,287.030,279.910,284.970,280.910,277.680,281.840,273.440,272.410,272.140,288.250,288.060,287.260,287.030,284.940,281.450,279.900,279.720,275.450,282.150,274.720,273.590,269.790,288.310,...,273.810,282.370,281.160,276.990,270.400,266.970,262.540,260.390,267.550,276.640,262.270,264.330,270.350,275.720,281.480,277.790,276.860,270.010,267.220,263.790,263.160,268.410,277.070,266.520,270.340,274.760,275.430,280.650,271.810,270.350,273.390,273.400,269.640,271.380,275.950,266.700,268.240,271.220,277.660,268.160
1421312400,288.590,288.720,288.350,287.770,287.750,285.070,287.150,286.710,286.120,284.610,281.030,277.310,277.680,288.460,288.470,288.120,287.580,287.250,285.560,285.780,284.580,283.550,284.240,280.740,278.340,276.630,288.490,288.580,287.950,287.200,286.290,285.610,284.100,283.030,282.920,283.790,280.420,278.710,275.680,288.110,...,281.250,288.620,288.300,281.120,273.070,272.590,271.730,272.240,273.840,282.530,269.540,270.710,276.050,282.210,288.120,281.950,280.680,273.890,273.430,271.820,272.730,274.930,282.940,273.280,276.850,280.070,280.600,285.840,273.960,274.760,278.180,276.530,276.650,276.710,281.150,271.040,273.530,276.590,282.330,274.720
1421323200,288.790,288.860,288.780,288.050,287.760,284.530,287.350,287.020,286.350,284.960,282.760,278.630,281.240,288.850,288.640,288.710,287.890,287.710,285.970,286.090,285.180,284.710,285.100,282.630,279.870,278.610,288.830,288.920,288.510,287.850,287.400,286.970,285.760,284.060,284.510,285.030,281.980,280.830,276.370,288.690,...,282.060,287.800,289.430,280.400,272.980,272.420,270.110,269.180,272.440,283.500,268.900,269.980,275.560,282.490,289.090,281.230,279.850,273.740,272.690,270.760,269.600,273.250,283.180,273.170,276.290,280.120,281.020,286.490,273.830,274.240,277.690,276.920,274.620,275.510,281.340,270.310,273.390,275.730,283.170,272.940
1421334000,288.990,288.870,288.720,287.890,287.700,283.380,287.130,286.560,285.

In [0]:
lss=pd.read_csv("drive/My Drive/IE 492/lasso_coeffs_new.csv")
lss=lss[['loc','coef']]
lss=lss[2:]
lss=lss.reset_index()[['loc','coef']]
lss

,loc,coef
0,temp.26.0_36.0,0.000000
1,temp.26.0_36.5,0.000000
2,temp.26.0_37.0,0.000000
3,temp.26.0_37.5,0.000000
4,temp.26.0_38.0,0.000000
...,...,...
502,temp.45.0_40.0,0.000000
503,temp.45.0_40.5,0.000000
504,temp.45.0_41.0,0.000000
505,temp.45.0_41.5,40.536716


In [0]:
cat_df_1

,Datetime,is_religious_holiday,is_normal_holiday,is_special_day,Hour,Day,Month,weekday
0,2015-12-31 00:00:00,0,1,0,0,31,12,3
1,2015-12-31 01:00:00,0,1,0,1,31,12,3
2,2015-12-31 02:00:00,0,1,0,2,31,12,3
3,2015-12-31 03:00:00,0,1,0,3,31,12,3
4,2015-12-31 04:00:00,0,1,0,4,31,12,3
...,...,...,...,...,...,...,...,...
36571,2020-03-03 19:00:00,0,0,0,19,3,3,1
36572,2020-03-03 20:00:00,0,0,0,20,3,3,1
36573,2020-03-03 21:00:00,0,0,0,21,3,3,1
36574,2020-03-03 22:00:00,0,0,0,22,3,3,1


In [0]:
cat_df_copy = cat_df_1.copy()
cat_df_copy = cat_df_copy.loc[cat_df_copy["Datetime"]<pd.to_datetime("2020-02-25 00:00:00")]


In [0]:
cat_df_copy

,Datetime,is_religious_holiday,is_normal_holiday,is_special_day,Hour,Day,Month,weekday,forecast_epoch,temp
0,2015-12-31 00:00:00,0,1,0,0,31,12,3,1.451520e+09,-2942.725379
1,2015-12-31 01:00:00,0,1,0,1,31,12,3,NaN,NaN
2,2015-12-31 02:00:00,0,1,0,2,31,12,3,NaN,NaN
3,2015-12-31 03:00:00,0,1,0,3,31,12,3,1.451531e+09,-3014.597932
4,2015-12-31 04:00:00,0,1,0,4,31,12,3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
36379,2020-02-24 19:00:00,0,0,0,19,24,2,0,1.582571e+09,-2127.401289
36380,2020-02-24 20:00:00,0,0,0,20,24,2,0,1.582574e+09,-2175.755296
36381,2020-02-24 21:00:00,0,0,0,21,24,2,0,1.582578e+09,-2191.230845
36382,2020-02-24 22:00:00,0,0,0,22,24,2,0,1.582582e+09,-2209.293316


In [0]:
weathers_pivoted_mat=weathers_pivoted.values
lss_mat=lss['coef'].values
dat={'forecast_epoch':weathers_pivoted.index, 'temp':np.dot(weathers_pivoted_mat,lss_mat)}
heat_comb=pd.DataFrame(dat)
heat_usable=weathers_pivoted.iloc[:,lss.loc[lss['coef']!=0].index]
heat_comb['Datetime']=pd.to_datetime(heat_comb['forecast_epoch'], unit='s')
cat_df_copy=pd.merge(cat_df_copy,heat_comb,how="left",on="Datetime",)

#cat_df_1['temp'].fillna(method ='ffill', inplace = True)
#cat_df_1=cat_df_1.drop(columns=['forecast_epoch'])



In [0]:
cat_df_copy = cat_df_copy.drop(columns = "forecast_epoch")
cat_df_copy["temp"] = cat_df_copy["temp"].interpolate()

In [0]:
cat_df_copy

,Datetime,is_religious_holiday,is_normal_holiday,is_special_day,Hour,Day,Month,weekday,temp
0,2015-12-31 00:00:00,0,1,0,0,31,12,3,-2942.725379
1,2015-12-31 01:00:00,0,1,0,1,31,12,3,-2966.682897
2,2015-12-31 02:00:00,0,1,0,2,31,12,3,-2990.640414
3,2015-12-31 03:00:00,0,1,0,3,31,12,3,-3014.597932
4,2015-12-31 04:00:00,0,1,0,4,31,12,3,-2911.907736
...,...,...,...,...,...,...,...,...,...
36379,2020-02-24 19:00:00,0,0,0,19,24,2,0,-2127.401289
36380,2020-02-24 20:00:00,0,0,0,20,24,2,0,-2175.755296
36381,2020-02-24 21:00:00,0,0,0,21,24,2,0,-2191.230845
36382,2020-02-24 22:00:00,0,0,0,22,24,2,0,-2209.293316


In [0]:
preds_copy = preds.copy()

In [0]:
preds_copy = preds_copy.loc[preds_copy["Datetime"]<pd.to_datetime("2020-02-25 00:00:00")]

In [0]:
preds_copy

,Datetime,LR_2,naive_2,LR_2_7_w_date,naive_7,LR_2_7,LR_7,ygur_rima_preds_datetime,Cat1_48hour,Cat2_categorical_48hour,Keras_Basic_dense_48,HW_26.05_15.17_48,reel,best_model,chosen,var_models,mean_models,index,primary_tree_predicted,tree_predicted_2
0,2019-02-01 00:00:00,31744.801869,31427.68,31374.377264,32021.03,31872.179211,32126.739766,31948.68315,31312.281942,31368.284964,31493.430,30684.197615,31878.29,LR_2_7,unknown,94693.466377,31668.948816,0,unknown,unknown
1,2019-02-01 01:00:00,30353.806407,29554.05,29685.964658,30105.80,30060.616031,30503.032726,29979.44233,29321.171197,29342.353761,29617.531,27926.819198,30064.08,LR_2_7,unknown,168073.850322,29852.376811,1,unknown,unknown
2,2019-02-01 02:00:00,29429.213104,28308.65,28505.461085,28730.71,28791.507529,29337.249341,28720.68574,28112.010835,28313.998802,28455.043,26546.141624,28813.77,LR_2_7,unknown,187176.127513,28670.452944,2,unknown,unknown
3,2019-02-01 03:00:00,28904.205359,27601.48,27855.221933,27957.44,28075.676511,28681.681110,28046.44589,27611.726344,27380.407098,27712.234,25968.405327,27865.76,LR_2_7_w_date,unknown,231800.259055,27982.651825,3,unknown,unknown
4,2019-02-01 04:00:00,28714.921348,27346.52,27727.815649,27802.84,27896.662105,28550.613245,27738.71164,27145.591865,27480.197660,27759.875,25889.936371,27554.48,Cat2_categorical_48hour,unknown,239324.011068,27816.374851,4,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9331,2020-02-24 19:00:00,37195.447912,38769.54,39977.124230,40182.15,39388.639462,39045.630965,40460.96518,38631.942945,38505.836340,39833.120,40097.851066,39591.24,LR_2_7,ygur_rima_preds_datetime,958544.125013,39199.039703,9331,LR_2_7_w_date,LR_2_7_w_date
9332,2020-02-24 20:00:00,36221.670166,37457.89,39189.688921,39403.06,38478.042832,38385.128615,39137.31917,38306.072723,38275.513006,38650.790,38974.698507,38806.87,Keras_Basic_dense_48,ygur_rima_preds_datetime,875012.367417,38350.517543,9332,naive_7,LR_2_7_w_date
9333,2020-02-24 21:00:00,35636.282484,36669.39,38252.574549,38236.59,37486.169249,37396.210547,38244.69382,36900.181936,36928.897077,37487.094,37849.274085,37548.92,Keras_Basic_dense_48,ygur_rima_preds_datetime,690388.173389,37323.808366,9333,LR_2_7_w_date,LR_2_7_w_date
9334,2020-02-24 22:00:00,34706.143404,35416.52,37098.995728,36903.27,36241.293133,36265.839224,36906.74632,36410.591260,36176.403391,36123.992,36452.470548,36354.90,Cat1_48hour,ygur_rima_preds_datetime,522022.634103,36224.979446,9334,Keras_Basic_dense_48,naive_7


index,variance,mean,date,temperature


In [0]:
predicted_model_v3=["unknown"]*shift_timespan
for i in tqdm(range(shift_timespan,len(cat_df_copy))):
    train_df=preds[["Datetime","best_model",'var_models','mean_models','index']].iloc[:i+48]
    train_df=pd.merge(train_df,cat_df_copy,how="left",on="Datetime")
    valid_df=train_df.iloc[-1]
    train_df=train_df.iloc[:i]
    tree=DecisionTreeClassifier()
    tree.fit(train_df.drop(columns=["best_model","Datetime"]),train_df["best_model"])
    # random forestt rreggrressor
    pred=tree.predict(pd.DataFrame(valid_df).T.drop(columns=["best_model","Datetime"]))
    # predict proba
    predicted_model_v3.append(pred[0])


ValueError: ignored

In [0]:
preds_copy["tree_predicted_3"]=predicted_model_v3
mapes_3=preds_copy.loc[preds_copy["tree_predicted_3"]!="unknown"].apply(lambda x: mean_absolute_percentage_error(x["reel"],x[x["tree_predicted_3"]]),axis=1)
mapes_3.mean()

##KUBİ WINDOW TREE ve diğerleri


In [0]:
train_df=pd.merge(preds.drop(columns=["chosen"]),cat_df_1,on="Datetime",how="left")

In [0]:
train_df

,Datetime,LR_2,naive_2,LR_2_7_w_date,naive_7,LR_2_7,LR_7,ygur_rima_preds_datetime,Cat1_48hour,Cat2_categorical_48hour,Keras_Basic_dense_48,HW_26.05_15.17_48,reel,best_model,is_religious_holiday,is_normal_holiday,is_special_day,Hour,Day,Month,weekday
0,2019-02-01 00:00:00,31744.801869,31427.68,31374.377264,32021.03,31872.179211,32126.739766,31948.68315,31312.281942,31368.284964,31493.430,30684.197615,31878.29,LR_2_7,0,0,0,0,1,2,4
1,2019-02-01 01:00:00,30353.806407,29554.05,29685.964658,30105.80,30060.616031,30503.032726,29979.44233,29321.171197,29342.353761,29617.531,27926.819198,30064.08,LR_2_7,0,0,0,1,1,2,4
2,2019-02-01 02:00:00,29429.213104,28308.65,28505.461085,28730.71,28791.507529,29337.249341,28720.68574,28112.010835,28313.998802,28455.043,26546.141624,28813.77,LR_2_7,0,0,0,2,1,2,4
3,2019-02-01 03:00:00,28904.205359,27601.48,27855.221933,27957.44,28075.676511,28681.681110,28046.44589,27611.726344,27380.407098,27712.234,25968.405327,27865.76,LR_2_7_w_date,0,0,0,3,1,2,4
4,2019-02-01 04:00:00,28714.921348,27346.52,27727.815649,27802.84,27896.662105,28550.613245,27738.71164,27145.591865,27480.197660,27759.875,25889.936371,27554.48,Cat2_categorical_48hour,0,0,0,4,1,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9523,2020-03-03 19:00:00,34433.450000,35049.21,38763.630766,39283.70,37640.730230,38283.936762,39055.31804,36533.611538,37024.468681,38331.830,38763.053506,37936.79,LR_2_7,0,0,0,19,3,3,1
9524,2020-03-03 20:00:00,34324.041524,34901.84,38428.945844,38791.40,37280.720755,37866.571236,38426.28253,36400.613389,37347.225005,37396.150,38194.641721,37553.37,Keras_Basic_dense_48,0,0,0,20,3,3,1
9525,2020-03-03 21:00:00,33821.654912,34225.14,37491.797795,37588.86,36301.219739,36847.073493,37493.64405,35472.911196,36789.700181,36533.484,37232.506117,36581.73,Keras_Basic_dense_48,0,0,0,21,3,3,1
9526,2020-03-03 22:00:00,33102.945835,33257.06,36424.989624,36290.64,35168.705158,35746.459492,36200.27294,34919.122612,35609.855302,35341.290,35950.473223,35434.36,Keras_Basic_dense_48,0,0,0,22,3,3,1


In [0]:
from sklearn.tree import DecisionTreeClassifier

def dt_window_approach(n,buffer):
    ## n'den baslayıp surkeli growing olarak gidiyor windows
    shift_timespan=n*24
    print("decision tree window approach with n = {}, buffer = {}".format(n,buffer))
    predicted_model=["unknown"]*(shift_timespan+buffer)
    ranking=["unknown"]*(shift_timespan+buffer)

    for i in tqdm(range(shift_timespan,len(preds)-buffer)):
        # buffer koy
        train_df_epoch=train_df.iloc[:i].reset_index()
        valid_df=train_df.iloc[i+buffer:i+buffer+1].reset_index()
        tree=DecisionTreeClassifier(max_depth=8,min_samples_leaf=10)
        tree.fit(train_df.drop(columns=["best_model","Datetime"]),train_df["best_model"])
        # random forestt rreggrressor

        pred=tree.predict(pd.DataFrame(valid_df).drop(columns=["best_model","Datetime"]))
        a=tree.classes_.tolist()
        b=tree.predict_proba(pd.DataFrame(valid_df).drop(columns=["best_model","Datetime"])).tolist()[0]
        ranking.append(dict(zip(a,b)))
        # predict proba
        predicted_model.append(pred[0])
    preds["tree_{}_n_{}_bufferpredicted".format(n,buffer)]=predicted_model
    return predicted_model,ranking

In [0]:
from sklearn.tree import DecisionTreeClassifier

def dt_window_approach(n,buffer):
    ## n'den baslayıp surkeli growing olarak gidiyor windows
    shift_timespan=n*24
    print("decision tree window approach with n = {}, buffer = {}".format(n,buffer))
    predicted_model=["unknown"]*(shift_timespan+buffer)
    ranking=["unknown"]*(shift_timespan+buffer)

    for i in tqdm(range(shift_timespan,len(preds)-buffer)):
        # buffer koy
        train_df_epoch=train_df.iloc[:i].reset_index()
        valid_df=train_df.iloc[i+buffer:i+buffer+1].reset_index()
        tree=DecisionTreeClassifier(max_depth=8,min_samples_leaf=10)
        tree.fit(train_df.drop(columns=["best_model","Datetime"]),train_df["best_model"])
        # random forestt rreggrressor

        pred=tree.predict(pd.DataFrame(valid_df).drop(columns=["best_model","Datetime"]))
        #a=tree.classes_.tolist()
        #b=tree.predict_proba(pd.DataFrame(valid_df).drop(columns=["best_model","Datetime"])).tolist()[0]
        #ranking.append(dict(zip(a,b)))
        # predict proba
        #predicted_model.append(pred[0])
    preds["tree_{}_n_{}_bufferpredicted".format(n,buffer)]=predicted_model
    return predicted_model,ranking

dt_window_approach(7,24)

decision tree window approach with n = 7, buffer = 24


ValueError: ignored

In [0]:
combins=np.concatenate([[(j,i) for i in [24,48]] for j in [7,14,28]])
rankings={}
predictions_={}
for n,buffer in combins:
    pred_model_list,ranking=dt_window_approach(n,buffer)
    rankings["n_{}_buffer_{}".format(n,buffer)]=ranking
    predictions_["n_{}_buffer_{}".format(n,buffer)]=pred_model_list

decision tree window approach with n = 7, buffer = 24


ValueError: ignored

In [0]:
preds

,Datetime,LR_2,naive_2,LR_2_7_w_date,naive_7,LR_2_7,LR_7,ygur_rima_preds_datetime,Cat1_48hour,Cat2_categorical_48hour,Cat1_24hour,Cat2_categorical_24hour,Keras_Basic_dense_24,Keras_Basic_dense_48,HW_preds_48hour,HW_preds_24hour,reel,best_model,chosen,tree_7_n_24_bufferpredicted,tree_7_n_48_bufferpredicted,tree_14_n_24_bufferpredicted,tree_14_n_48_bufferpredicted
0,2019-02-01 00:00:00,31744.801869,31427.68,31374.377264,32021.03,31872.179211,32126.739766,31948.68315,31312.281942,31368.284964,31339.433448,31421.093027,31503.037,31493.430,31161.780355,32245.453202,31878.29,LR_2_7,unknown,unknown,unknown,unknown,unknown
1,2019-02-01 01:00:00,30353.806407,29554.05,29685.964658,30105.80,30060.616031,30503.032726,29979.44233,29321.171197,29342.353761,29885.517565,29421.283832,29815.424,29617.531,29447.620186,30317.336597,30064.08,LR_2_7,unknown,unknown,unknown,unknown,unknown
2,2019-02-01 02:00:00,29429.213104,28308.65,28505.461085,28730.71,28791.507529,29337.249341,28720.68574,28112.010835,28313.998802,28391.046167,28539.136838,28635.771,28455.043,28451.395711,28718.710641,28813.77,LR_2_7,unknown,unknown,unknown,unknown,unknown
3,2019-02-01 03:00:00,28904.205359,27601.48,27855.221933,27957.44,28075.676511,28681.681110,28046.44589,27611.726344,27380.407098,27894.855316,27672.298537,27855.768,27712.234,27704.391751,27928.743341,27865.76,Keras_Basic_dense_24,unknown,unknown,unknown,unknown,unknown
4,2019-02-01 04:00:00,28714.921348,27346.52,27727.815649,27802.84,27896.662105,28550.613245,27738.71164,27145.591865,27480.197660,27416.535458,27332.009369,27631.098,27759.875,27392.449683,27679.824148,27554.48,Cat2_categorical_48hour,unknown,unknown,unknown,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9523,2020-03-03 19:00:00,34433.450000,35049.21,38763.630766,39283.70,37640.730230,38283.936762,39055.31804,36533.611538,37024.468681,36842.214179,36960.207088,37731.016,38331.830,35196.746675,39194.639686,37936.79,Keras_Basic_dense_24,ygur_rima_preds_datetime,Keras_Basic_dense_24,Keras_Basic_dense_48,Keras_Basic_dense_24,Keras_Basic_dense_48
9524,2020-03-03 20:00:00,34324.041524,34901.84,38428.945844,38791.40,37280.720755,37866.571236,38426.28253,36400.613389,37347.225005,37298.866743,37127.922048,38224.590,37396.150,34905.761724,38307.274402,37553.37,Keras_Basic_dense_48,ygur_rima_preds_datetime,Keras_Basic_dense_24,Keras_Basic_dense_48,Keras_Basic_dense_24,Keras_Basic_dense_48
9525,2020-03-03 21:00:00,33821.654912,34225.14,37491.797795,37588.86,36301.219739,36847.073493,37493.64405,35472.911196,36789.700181,36169.047983,36589.808965,37079.137,36533.484,34102.138172,37190.286189,36581.73,Cat2_categorical_24hour,ygur_rima_preds_datetime,Keras_Basic_dense_24,Keras_Basic_dense_48,Keras_Basic_dense_24,Keras_Basic_dense_48
9526,2020-03-03 22:00:00,33102.945835,33257.06,36424.989624,36290.64,35168.705158,35746.459492,36200.27294,34919.122612,35609.855302,35232.776345,35744.313864,35741.207,35341.290,33085.861894,35839.197807,35434.36,Keras_Basic_dense_48,ygur_rima_preds_datetime,LR_2_7_w_date,Keras_Basic_dense_48,LR_2_7_w_date,Keras_Basic_dense_48


In [0]:
fig=tree.fit(train_df.drop(columns=["best_model","Datetime"]),train_df["best_model"])
_, ax = plt.subplots(figsize=(15, 15))  # whatever size you want
plot_tree(fig,ax=ax,feature_names=train_df.drop(columns=["Datetime","best_model"]).columns,fontsize=16);

NameError: ignored

In [0]:
from sklearn.tree import DecisionTreeClassifier

shift_timespan=30*24
predicted_model=["unknown"]*shift_timespan
ranking=["unknown"]*shift_timespan
for i in tqdm(range(shift_timespan,len(preds),24)):
    # buffer koy
    train_df=preds[["Datetime","best_model"]].iloc[:i]
    train_df=pd.merge(train_df,cat_df_1,how="left",on="Datetime")
    valid_df=train_df.iloc[-24:]
    train_df=train_df.iloc[:i-24]
    tree=DecisionTreeClassifier(max_depth=16,min_samples_leaf=10)
    tree.fit(train_df.drop(columns=["best_model","Datetime"]),train_df["best_model"])
    # random forestt rreggrressor
    pred=tree.predict(pd.DataFrame(valid_df).drop(columns=["best_model","Datetime"]))
    # a=tree.classes_.tolist()
    # b=tree.predict_proba(pd.DataFrame(valid_df).T.drop(columns=["best_model","Datetime"])).tolist()[0]
    # ranking.append(dict(zip(a,b)))
    # predict proba
    predicted_model=np.concatenate((np.array(predicted_model),pred))
# preds["ranking"]=ranking
preds["tree_predicted"]=predicted_model
preds

,Datetime,LR_2,naive_2,LR_2_7_w_date,naive_7,LR_2_7,LR_7,ygur_rima_preds_datetime,Cat1_48hour,Cat2_categorical_48hour,Keras_Basic_dense_48,HW_26.05_15.17_48,reel,best_model,chosen,tree_predicted
0,2019-02-01 00:00:00,31744.801869,31427.68,31374.377264,32021.03,31872.179211,32126.739766,31948.68315,31312.281942,31368.284964,31493.430,30684.197615,31878.29,LR_2_7,unknown,unknown
1,2019-02-01 01:00:00,30353.806407,29554.05,29685.964658,30105.80,30060.616031,30503.032726,29979.44233,29321.171197,29342.353761,29617.531,27926.819198,30064.08,LR_2_7,unknown,unknown
2,2019-02-01 02:00:00,29429.213104,28308.65,28505.461085,28730.71,28791.507529,29337.249341,28720.68574,28112.010835,28313.998802,28455.043,26546.141624,28813.77,LR_2_7,unknown,unknown
3,2019-02-01 03:00:00,28904.205359,27601.48,27855.221933,27957.44,28075.676511,28681.681110,28046.44589,27611.726344,27380.407098,27712.234,25968.405327,27865.76,LR_2_7_w_date,unknown,unknown
4,2019-02-01 04:00:00,28714.921348,27346.52,27727.815649,27802.84,27896.662105,28550.613245,27738.71164,27145.591865,27480.197660,27759.875,25889.936371,27554.48,Cat2_categorical_48hour,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9523,2020-03-03 19:00:00,34433.450000,35049.21,38763.630766,39283.70,37640.730230,38283.936762,39055.31804,36533.611538,37024.468681,38331.830,38763.053506,37936.79,LR_2_7,ygur_rima_preds_datetime,Cat1_48hour
9524,2020-03-03 20:00:00,34324.041524,34901.84,38428.945844,38791.40,37280.720755,37866.571236,38426.28253,36400.613389,37347.225005,37396.150,38194.641721,37553.37,Keras_Basic_dense_48,ygur_rima_preds_datetime,Cat1_48hour
9525,2020-03-03 21:00:00,33821.654912,34225.14,37491.797795,37588.86,36301.219739,36847.073493,37493.64405,35472.911196,36789.700181,36533.484,37232.506117,36581.73,Keras_Basic_dense_48,ygur_rima_preds_datetime,Cat1_48hour
9526,2020-03-03 22:00:00,33102.945835,33257.06,36424.989624,36290.64,35168.705158,35746.459492,36200.27294,34919.122612,35609.855302,35341.290,35950.473223,35434.36,Keras_Basic_dense_48,ygur_rima_preds_datetime,Cat1_48hour


In [0]:
# parameterler
# tahminleri de eklemek
# gün bazlı seyi kacırıyoruz saat saat baktıgımız için, günün bütününe dair

In [0]:
2.540852726450188

In [0]:
mapes_tree={}
for col in [i for i in preds.columns if "tree" in i ]:
    mapes_=preds.loc[preds[col]!="unknown"].apply(lambda x: mean_absolute_percentage_error(x["reel"],x[x[col]]),axis=1)
    mapes_tree[col]=mapes_.mean()

In [0]:
mapes_tree

{'tree_predicted': 4.196852105402424}

In [0]:
preds.iloc[[8024]]

In [0]:
preds[preds["tree_predicted"]!="unknown"]["tree_predicted"].value_counts()

In [0]:
preds

In [0]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


In [0]:
from sklearn.metrics import confusion_matrix
for col in ["chosen","chosen_5","chosen_14","tree_predicted"]:
    pah_="drive/My Drive/colab_datas/IE 492/kubi_preds/eval_plots/conf_matrices/"+col
    if not os.path.exists(pah_):
        os.mkdir(pah_)
    unq_pred=preds.loc[preds[col]!="unknown"].reset_index(drop=True)
    clss=labels=unq_pred[col].unique().tolist()
    y_true=unq_pred["best_model"].values
    y_pred=unq_pred[col].values
    cnf_matrix = confusion_matrix(y_true, y_pred,labels=clss)
    np.set_printoptions(precision=2)
    plt.figure(figsize=(15,15))
    plot_confusion_matrix(cnf_matrix, classes=clss,
                        title='Confusion matrix of {}, without normalization'.format(col))
    plt.savefig(pah_+"/conf_matrix.png")


In [0]:
preds["best_case"]=preds.apply(lambda x:x[x["best_model"]],axis=1)
preds["window_case"]=preds.apply(lambda x:x[x["chosen"]] if x["chosen"]!="unknown" else 0,axis=1)
preds["tree_case"]=preds.apply(lambda x:x[x["tree_predicted"]] if x["tree_predicted"]!="unknown" else 0,axis=1)

In [0]:
for col in preds.columns[1:-5]:
    eval_function(col,preds["reel"],preds[col],preds)

In [0]:
eval_function("best_case",preds["reel"],preds["best_case"],preds)

In [0]:
pred_wo_unk=preds.loc[preds["window_case"]!=0].reset_index(drop=True)
eval_function("meta_window_case",pred_wo_unk["reel"],pred_wo_unk["window_case"],pred_wo_unk)

In [0]:
pred_wo_unk=preds.loc[preds["tree_case"]!=0].reset_index(drop=True)
eval_function("meta_tree_case",pred_wo_unk["reel"],pred_wo_unk["tree_case"],pred_wo_unk)

In [0]:
pd.DataFrame.from_dict(mapes,orient="index")